## Logistic Regression

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper

## Load the data
Note: documentation for the dataset is in the file data/adult.names

In [ ]:
df = pd.read_csv('data/adult.data',header=None)

# Explore the data

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
#Add a header row to df
df.columns = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
    "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
    "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
]

In [ ]:
df.head()

In [ ]:
#Check if there are any missing values
df.isnull().values.any()

In [ ]:
#Inspect the unique values in each column
for col in df.columns.values:
    uniq = df[col].unique()
    print(col,': ',df[col].unique())
    print(len(uniq)," unique values")
    print()

In [ ]:
#Inspect the unique values
template = "{} has {} unique values"
for col in df.columns.values:
    uniq = df[col].unique()
    print(template.format(col,len(uniq)))


In [ ]:
#Unique values of Income column. This is the target.
df['Income'].unique()

## Prepare the data

In [ ]:
#Convert "<=50K" to -1 and ">50K" to 1
df['Income'] = np.where(df['Income'] == ' >50K',1,0)

In [ ]:
df.head()

In [ ]:
#Descriptive statistics for the numerical columns
df.describe(include=np.number).round(2)

In [ ]:
#Descriptive statistics for the categorical columns
df.describe(include=object).round(2)

In [ ]:
#For features we will use Age, Relationship, ExucationNum, Gender, and Occupation
features = pd.DataFrame({'Age':df['Age'], 'Relationship':df['Relationship'], 'Educ':df['EducationNum'], \
'Gender':df['Gender'],'Occupation': df['Occupation']})
features

In [ ]:
#We scale Age and Educ columns
mapper = DataFrameMapper([\
(['Age'],StandardScaler()),\
(['Relationship'],None),\
(['Educ'],StandardScaler()),\
(['Gender'],None),\
(['Occupation'],None)\
],df_out=True)

In [ ]:
features=mapper.fit_transform(features)
features

In [ ]:
#Dummy encode the categorical data
features = pd.get_dummies(features,["Relationship","Gender","Hours"])
features

In [ ]:
#Split the data into a training set and test set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features, df['Income'], test_size=0.1, random_state=514)

## Create the model

In [ ]:
#Create the model
clf = LogisticRegression(random_state=514, solver='liblinear' ,max_iter=1000).fit(x_train, y_train)

# Evaluate the model

In [ ]:
# The confusion matrix for the test data
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
confusion_matrix(clf.predict(x_test),y_test)

In [ ]:
# Score against test data
clf.score(x_test, y_test)

### Could you get a better model using cross validation or a grid search?

In [ ]:
#Cross validation
from sklearn.model_selection import cross_val_score
model = LogisticRegression(random_state=514, solver='lbfgs', max_iter=1000, C=1)
print(cross_val_score(model, features, df['Income'], cv=5))

In [ ]:
#This takes about a minute to run
#Use GridSearchCV to find the best choice of solver from ‘newton-cg’, ‘lbfgs’, ‘liblinear’,
# and best value of C from 0.01,0.1,1,5,10,50,100
from sklearn.model_selection import GridSearchCV
model = LogisticRegression(random_state=514, max_iter = 1000)
grid = {'solver':['newton-cg','liblinear','lbfgs'],\
'C':[0.01,0.1,1,5,10,50,100]}
search = GridSearchCV(model,grid,scoring='accuracy', cv=5)
search.fit(features, df['Income'])
print(search.best_params_)
print(search.best_score_)

In [ ]:
best_model = search.best_estimator_

In [ ]:
#We calculate the roc curve
y_true=y_test.values
y_probs = clf.predict_proba(x_test)
y_probs
fpr, tpr,_=roc_curve(y_true, y_probs[:,1])
print("AUC is ", roc_auc_score(y_true, y_probs[:,1]))

In [ ]:
# Does the best_model have a better score than clf?

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
##Adding the ROC
plt.plot(fpr, tpr, color='red',
 lw=2, label='ROC curve')
##Random FPR and TPR
plt.plot([0, 1], [0, 1], color='blue', lw=2, linestyle='--')
##Title and label
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC curve')

In [ ]:
# Numbers calculated for each label (0 and 1)
# Macro avg is just an average and does not take data imbalance into account
# Weighted average is an average with weights depending on the frequency of the labels
from sklearn.metrics import classification_report
cr=classification_report(y_test, clf.predict(x_test))
print(cr)

In [ ]:
# What are the corresponding scores for best_model?